# Training a LoRA Adapter for React/JSX Widgets on Colab

Trains a LoRA adapter for Apple Foundation Models (v26.0.0) to specialize a chatbot in React/JSX widget generation using 434 hand-crafted widgets. Runs on Colab’s Linux GPU (T4/A100) with MLX CUDA backend. Matches Apple’s `end_to_end_example.ipynb` workflow.

## Prerequisites
- Colab GPU runtime (free T4 or Pro A100).
- Apple Developer Program toolkit in `/content/drive/MyDrive/toolkit/`.
- 434 widget files in `/content/drive/MyDrive/widgets/`.
- Hugging Face token.
- Python 3.11+.

## Step 1: Setup

Mount Drive, install dependencies, verify GPU, set memory optimization.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
sys.path.append(os.path.abspath('/content/drive/MyDrive/AITraining'))
sys.path.append(os.path.abspath('/content/drive/MyDrive/AITraining/examples'))

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/MyDrive/AITraining/requirements.txt
!pip install mlx[cuda] datasets pandas
!pip install flash-attn --no-build-isolation
!huggingface-cli login  # Enter token
!nvidia-smi  # Verify GPU

# Optimize memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'  # Disable MPS limit
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU


  Using cached nvidia_cublas_cu12-12.9.1.4-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.9.86-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
Using cached nvidia_cublas_cu12-12.9.1.4-py3-none-manylinux_2_27_x86_64.whl (581.2 MB)
Using cached nvidia_cuda_nvrtc_cu12-12.9.86-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl (89.6 MB)
  Attempting uninstall: nvidia-cuda-nvrtc-cu12
    Found existing installation: nvidia-cuda-nvrtc-cu12 12.6.77
    Uninstalling nvidia-cuda-nvrtc-cu12-12.6.77:
      Successfully uninstalled nvidia-cuda-nvrtc-cu12-12.6.77
  Attempting uninstall: nvidia-cublas-cu12
    Found existing installation: nvidia-cublas-cu12 12.6.4.1
    Uninstalling nvidia-cublas-cu12-12.6.4.1:
      Successfully uninstalled nvidia-cublas-cu12-12.6.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [ ]:
import sys
import os
import importlib.util

# Remove the problematic path additions
if '/content/drive/MyDrive/AITraining/examples' in sys.path:
    sys.path.remove('/content/drive/MyDrive/AITraining/examples')

# Add the parent directory to sys.path so Python can find the examples package
aitraining_path = '/content/drive/MyDrive/AITraining'
if aitraining_path not in sys.path:
    sys.path.insert(0, aitraining_path)

print("🔍 Updated sys.path:")
for i, path in enumerate(sys.path[:5]):  # Show first 5 paths
    print(f"  {i}: {path}")

# Now try importing the package properly
try:
    # Import the examples package
    import examples

    # Force reload to ensure fresh import
    importlib.reload(examples)

    print("✅ Examples package imported successfully")

    # Now try to import the submodules
    from examples import generate
    from examples import messages

    print("✅ Submodules imported successfully")

    # Finally, try the specific functions
    from examples.generate import generate_content, GenerationConfiguration
    from examples.messages import Message

    print("✅ Function imports successful!")
    print("🎉 All imports working!")

except ImportError as e:
    print(f"❌ Import failed: {e}")

    # Let's try a different approach - check what's in the __init__.py
    init_file = '/content/drive/MyDrive/AITraining/examples/__init__.py'
    with open(init_file, 'r') as f:
        init_content = f.read()
        print(f"\n🔍 __init__.py content:")
        print(init_content[:500] + "..." if len(init_content) > 500 else init_content)

🔍 Updated sys.path:
  0: /content
  1: /env/python
  2: /usr/lib/python312.zip
  3: /usr/lib/python3.12
  4: /usr/lib/python3.12/lib-dynload
✅ Examples package imported successfully
✅ Submodules imported successfully
✅ Function imports successful!
🎉 All imports working!


In [ ]:
import torch
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU type: {torch.cuda.get_device_name(0)}")

GPU memory: 0.00 GB
GPU type: NVIDIA A100-SXM4-80GB


## Step 2: Test Generation

Verify setup with base model.

In [ ]:
# These imports will be available in Colab after toolkit upload
try:
    from examples.generate import generate_content, GenerationConfiguration
    from examples.messages import Message
except ImportError:
    print("⚠️  Toolkit not yet uploaded to Google Drive")
    print("   These imports will work in Colab after upload")


# Force model to stay in memory
import torch
torch.cuda.empty_cache()  # Clear any fragmented memory first

# Then run generation - model should stay loaded
output = generate_content(
    [[Message.from_system('You are a helpful assistant generating React/JSX widgets.'),
      Message.from_user('Create a React button widget')]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=128)
)

print(f"After generation: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(output[0].response)

After generation: 0.01 GB
Certainly! Below is a simple example of a React button widget. This button will display a message when clicked and change its color based on the state of a boolean variable.

```jsx
import React, { useState } from 'react';

const ButtonWidget = () => {
  const [isActive, setIsActive] = useState(false);

  const handleClick = () => {
    setIsActive(!isActive);
  };

  return (
    <div>
      <button
        style={{
          backgroundColor: isActive ? '


## Step 3: Train Adapter



Select dataset


In [13]:
#DATA_SET = 'test_complete'
#DATA_SET = 'withtoolcall'
#DATA_SET = 'toolcall_plus_notruncation'
#DATA_SET = 'with_tool_calls_4'
DATA_SET = 'fixed_create_directive_bias'



Run the training





In [14]:
TRAIN_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/train.jsonl'
VALID_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/valid.jsonl'
# TEST_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/test.jsonl'

from examples.train_adapter import AdapterTrainingConfiguration, train_adapter

config1=AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=1,  # Reduce from 4 to 1
    gradient_accumulation_steps=8,  # Compensate by accumulating gradients
    enable_activation_checkpointing=True,
    max_sequence_length=256,  # Reduce further
    compile_model=False  # Disable compilation
)

config2 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=512,  # Increase for better context
    compile_model=False
)

config2 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=512,  # Increase for better context
    compile_model=False
)

config3 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=4095,
    compile_model=False
)

train_adapter(
    train_data=TRAIN_FILE,
    eval_data=VALID_FILE,
    config=config3,
    checkpoint_dir='/content/drive/MyDrive/checkpoints'
)

INFO:examples.utils:Fine-tuning adapters with configuration: 
AdapterTrainingConfiguration(epochs=6, learning_rate=0.0001, batch_size=2, linear_warmup_epochs=1, gradient_accumulation_steps=4, enable_activation_checkpointing=True, precision='bf16-mixed', compile_model=False, weight_decay=0.01, clip_grad_norm=1.0, max_sequence_length=4095, fixed_sized_sequences=False, pack_sequences=False, loss_update_frequency=3)
INFO:examples.utils:Loading base model on cuda with precision torch.float32
INFO:examples.utils:Total parameters 3178001792
INFO:examples.utils:Total trainable parameters 66633728
INFO:examples.utils:Gradient scaling is enabled: False
INFO:examples.utils:Epoch 1/6
Evaluation: 100%|==========| 3/3 [00:01<00:00,  2.02it/s, loss=0.842]
INFO:examples.utils:Epoch 2/6
Evaluation: 100%|==========| 3/3 [00:01<00:00,  2.08it/s, loss=0.809]
INFO:examples.utils:Epoch 3/6
Evaluation: 100%|==========| 3/3 [00:01<00:00,  2.08it/s, loss=0.798]
INFO:examples.utils:Epoch 4/6
Evaluation: 100%|==

## Step 5: Train Draft Model (Optional)

Train draft model for faster inference.

In [ ]:
from examples.train_draft_model import DraftModelTrainingConfiguration, train_draft_model

train_draft_model(
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
    train_data=TRAIN_FILE,
    eval_data=VALID_FILE,
    config=DraftModelTrainingConfiguration(
        epochs=3,
        learning_rate=1e-4
        ),
    checkpoint_dir='/content/drive/MyDrive/checkpoints',
    checkpoint_frequency=1
)

INFO:examples.utils:Loading target (/teacher) model on cuda with precision torch.float32
INFO:examples.utils:Loading draft (/student) model on cuda with precision torch.float32
INFO:examples.utils:Gradient scaling is enabled: False
INFO:examples.utils:Epoch 1/3
Training:   0%|          | 0/15 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.24 GiB. GPU 0 has a total capacity of 79.32 GiB of which 15.18 GiB is free. Process 204572 has 64.13 GiB memory in use. Of the allocated memory 63.29 GiB is allocated by PyTorch, and 343.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Step 6: Evaluate Adapter

Test adapter quality.

In [15]:
from examples.generate import generate_content, GenerationConfiguration
from examples.messages import Message

output = generate_content(
    [[Message.from_system('You are a helpful assistant generating Übersicht widgets.'),
      Message.from_user('Generate a widget for a toggle switch with customizable colors')]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=128),
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
#    draft_checkpoint='/content/drive/MyDrive/checkpoints/draft-model-final.pt'
)
print(output[0].response)

<executable_start> ```function
[{"name": "widget:get_widget_image", "arguments": {"image_url": "https://www.widget-maker.com/wp-content/uploads/2021/01/toggle-switch-widget.png"}}]```<executable_end><turn_end>


## Step 7: Export Adapter

Export to `.fmadapter`.

In [16]:
from export.export_fmadapter import Metadata, export_fmadapter

metadata = Metadata(
    author='Your Name',
    description='Adapter for generating Übersicht widgets'
)

export_fmadapter(
    output_dir='/content/drive/MyDrive/AITraining/US_E2E/TrainedAdapter',
    adapter_name='uebersicht_widgets',
    metadata=metadata,
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
#    draft_checkpoint='/content/drive/MyDrive/checkpoints/draft-model-final.pt'
)

'/content/drive/MyDrive/AITraining/US_E2E/TrainedAdapter/uebersicht_widgets.fmadapter'

## Notes
- Upload toolkit/widgets to `/content/drive/MyDrive/toolkit/` and `/widgets/`.
- Update `prompt_map` for all 434 widgets.
- Training: ~30–60 min on T4 (free); ~15–30 min on A100 (Pro, $10/mo).
- Memory: `PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0` mitigates MPS errors; T4 has 16GB VRAM.
- If `tamm` fails, contact Apple Developer Support.
- Run `produce_asset_pack` on MacOS; verify `.fmadapter` locally.